### 读取数据

In [1]:
import pandas as pd 

filename = './data/sqlResult_1558435.csv'
chunk = pd.read_csv(filename, encoding='gb18030', chunksize=1000)
content = pd.concat(chunk)

### 处理数据，清洗

In [2]:
import re

def clean(string): return re.findall('\w+', string)

articles = content['content'].tolist()
articles_clean = [''.join(clean(str(s))) for s in articles]

### 处理数据，缓存构建词语表

In [4]:
with open('./data/article_temp.txt', 'w') as f:
    for a in articles_clean:
        f.write(a + '\n')
        
import jieba
def cut(string): return list(jieba.cut(string))

TOKENS = []
for i, line in enumerate(open('./data/article_temp.txt')):
    if i > 10000: break
    TOKENS += cut(line)

from collections import Counter
words_count = Counter(TOKENS)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Think\AppData\Local\Temp\jieba.cache
Loading model cost 1.510 seconds.
Prefix dict has been built succesfully.


### 一元语言模型

In [5]:
def prob_1(word):
    return words_count[word] / len(TOKENS)
prob_1('我们')

0.001554473157589251

### 二元语言模型

In [6]:
TOKENS = [str(t) for t in TOKENS]
TOKENS_2_GRAM = [''.join(TOKENS[i:i + 2]) for i in range(len(TOKENS[:-2]))]
words_count_2 = Counter(TOKENS_2_GRAM)
def prob_2(word1, word2):
    if word1 + word2 in words_count_2: return words_count_2[word1 + word2] / len(TOKENS_2_GRAM)
    else: return 1 / len(TOKENS_2_GRAM)
prob_2('我们', '在')

4.683455274626447e-05

### 使用二元语言模型评估句子

In [7]:
def get_probablity(sentence):
    words = cut(sentence)
    sentence_pro = 1 
    for i, word in enumerate(words[:-1]):
        next_ = words[i + 1];
        probability = prob_2(word, next_)
        sentence_pro *= probability
    return sentence_pro
get_probablity('小明今天抽奖抽了一台苹果手机')

2.7282356776710467e-44